In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from src.loader import select_data_c02
from os import path

In [6]:
MESES = [mes for mes in range(202001,202108,1)]

In [36]:
# CONECTAR A BIGQUERY

ganancia_acierto = 780000
costo_estimulo = 20000

data = select_data_c02([202101,202102,202103,202104,202105,202106,202108])

In [37]:

mes_train = [202101,202102,202103,202104,202105,202106]
mes_score = 202108
data = data.to_pandas()


In [38]:
train_data = data[data['foto_mes'].isin(mes_train)]
score_data = data[data['foto_mes'] == mes_score]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted


,Train Null Percentage,Score Null Percentage,diff
Master_mconsumospesos,59.503352,57.971023,1.532328
Master_mconsumosdolares,59.503352,57.971023,1.532328
Master_madelantopesos,59.503352,57.971023,1.532328
Master_madelantodolares,59.503352,57.971023,1.532328
Master_mpagospesos,59.503352,57.971023,1.532328
...,...,...,...
mttarjeta_visa_debitos_automaticos,0.000000,0.000000,0.000000
ctarjeta_master_debitos_automaticos,0.000000,0.000000,0.000000
mttarjeta_master_debitos_automaticos,0.000000,0.000000,0.000000
cpagodeservicios,0.000000,0.000000,0.000000


In [39]:

train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)
diff_zero_percentage_sorted


,Train Zero Percentage,Score Zero Percentage,diff_zero_percentage
Master_fultimo_cierre,11.580385,0.000000,11.580385
Visa_fultimo_cierre,11.556673,0.000000,11.556673
Visa_mpagado,80.479928,68.961668,11.518260
cmobile_app_trx,27.985291,20.736916,7.248375
ctransferencias_recibidas,27.100719,23.291187,3.809532
...,...,...,...
Master_fechaalta,0.000000,0.000000,0.000000
Visa_Fvencimiento,0.000000,0.000000,0.000000
ccaja_ahorro,0.000000,0.000000,0.000000
Visa_mlimitecompra,0.000000,0.000000,0.000000


In [45]:

def psi_orig(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null



buckets=10
column = 'mprestamos_personales'
expected = train_data[column]
actual = score_data[column]

def psi_formula(expected_prop, actual_prop):
    result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
    return result

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

expected_prop = expected_counts / len(expected_not_null)
actual_prop = actual_counts / len(actual_not_null)

psi_not_null = psi_formula(expected_prop, actual_prop).sum()

psi_null = 0

psi_value = psi(expected, actual)



In [62]:
import numpy as np
import pandas as pd

def psi(expected, actual, buckets=10):
    def psi_formula(expected_prop, actual_prop):
        # epsilon para evitar log(0)
        eps = 1e-6
        expected_prop = np.where(expected_prop == 0, eps, expected_prop)
        actual_prop   = np.where(actual_prop == 0, eps, actual_prop)
        return (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)

    # Quitamos nulos sólo para el cálculo de bins
    expected_not_null = expected.dropna()
    actual_not_null   = actual.dropna()

    # Si no hay datos suficientes, devolvemos 0 para no romper
    if len(expected_not_null) == 0 or len(actual_not_null) == 0:
        return 0.0

    # qcut con retbins=True: nos da los bins explícitos
    try:
        _, breakpoints = pd.qcut(
            expected_not_null,
            q=buckets,
            retbins=True,
            duplicates='drop'
        )
    except ValueError:
        # Por si hay muy pocos valores distintos y qcut no puede armar bins
        return 0.0

    # Nos aseguramos de que los breakpoints sean únicos y ordenados
    breakpoints = np.unique(breakpoints)

    # Si hay menos de 2 puntos de corte, no se puede armar histograma
    if len(breakpoints) < 2:
        return 0.0

    # Conteos en cada bin
    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _   = np.histogram(actual_not_null,  bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop   = actual_counts   / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    # ---- componente de nulos ----
    psi_null = 0.0
    if expected.isnull().any() or actual.isnull().any():
        expected_null_percentage = expected.isnull().mean()
        actual_null_percentage   = actual.isnull().mean()
        psi_null = psi_formula(
            np.array([expected_null_percentage]),
            np.array([actual_null_percentage])
        ).sum()

    return float(psi_not_null + psi_null)


In [63]:
def calcular_psi(train_data, score_data):
    psi_results = []
    for column in train_data.columns:
        if column not in [
            'foto_mes', 'clase_ternaria',
            'ctarjeta_master_descuentos','mtarjeta_master_descuentos',
            'mtarjeta_visa_descuentos','ctarjeta_visa_descuentos',
            'ccajeros_propios_descuentos','mcajeros_propios_descuentos'
        ]:
            train_variable  = train_data[column]
            score_variable  = score_data[column]
            psi_value       = psi(train_variable, score_variable)
            psi_results.append({'feature': column, 'psi': psi_value})

    psi_df = pd.DataFrame(psi_results)
    psi_df = psi_df.sort_values('psi', ascending=False)
    psi_df = psi_df[psi_df.psi > 0.1]

    return psi_df

psi_mes = {}
for mes in train_data.foto_mes.unique():
    d_train = train_data.loc[train_data.foto_mes == mes]
    psi_mes[mes] = calcular_psi(d_train, score_data)


In [75]:
def calcular_psi(train_data, score_data):
    psi_results = []
    print(f' Calculo PSI para {train_data.foto_mes.unique()}')
    for column in train_data.columns:
      if column not in ['foto_mes', 'clase_ternaria', 'ctarjeta_master_descuentos','mtarjeta_master_descuentos','mtarjeta_visa_descuentos','ctarjeta_visa_descuentos','ccajeros_propios_descuentos','mcajeros_propios_descuentos']:
        #print(column)
        if train_data[column].dtype.kind not in "iufc":
            print(f'columna {column} no es un int')
            continue

        train_variable = train_data[column]
        score_variable = score_data[column]
        psi_value = psi(train_variable, score_variable)
        psi_results.append({'feature': column, 'psi': psi_value})

    psi_df = pd.DataFrame(psi_results)
    psi_df = psi_df.sort_values('psi', ascending=False)
    psi_df = psi_df[psi_df.psi > 0.1]

    return psi_df

In [76]:
psi_mes = {}
for mes in train_data.foto_mes.unique():
    print(mes)
    d_train = train_data.loc[train_data.foto_mes == mes]
    psi_mes[mes] = calcular_psi(d_train, score_data)
    print(psi_mes[mes])


202104
 Calculo PSI para [202104]
                 feature       psi
129     Visa_Finiciomora  1.698006
106  Master_Fvencimiento  0.100316
202102
 Calculo PSI para [202102]
                       feature        psi
116      Master_fultimo_cierre  11.431339
138        Visa_fultimo_cierre  10.629256
129           Visa_Finiciomora   5.367363
65   mcomisiones_mantenimiento   0.551858
107         Master_Finiciomora   0.366856
106        Master_Fvencimiento   0.269267
8                  mcomisiones   0.224540
67           mcomisiones_otras   0.223878
139               Visa_mpagado   0.141564
6                mrentabilidad   0.129446
202101
 Calculo PSI para [202101]
                       feature       psi
65   mcomisiones_mantenimiento  0.551240
106        Master_Fvencimiento  0.542862
107         Master_Finiciomora  0.331485
8                  mcomisiones  0.118697
67           mcomisiones_otras  0.118475
74   mtransferencias_recibidas  0.103729
202106
 Calculo PSI para [202106]
          

In [74]:
array_df = []
for k,v in psi_mes.items():
    v['mes'] = k
    array_df.append(v)
array_df = pd.concat(array_df)

In [71]:
pd.pivot_table(array_df, index='feature', columns='mes', values='psi')

mes,202101,202102,202103,202104,202106
feature,,,,,
Master_Finiciomora,0.331485,0.366856,0.863989,NaN,3.145861
Master_Fvencimiento,0.542862,0.269267,0.181100,0.100316,NaN
Master_fultimo_cierre,NaN,11.431339,NaN,NaN,NaN
Visa_Finiciomora,NaN,5.367363,1.741151,1.698006,3.489909
Visa_fultimo_cierre,NaN,10.629256,NaN,NaN,NaN
Visa_mpagado,NaN,0.141564,NaN,NaN,0.197575
cpayroll_trx,NaN,NaN,NaN,NaN,0.229340
mcomisiones,0.118697,0.224540,NaN,NaN,NaN
mcomisiones_mantenimiento,0.551240,0.551858,NaN,NaN,NaN


In [ ]:
['Master_Finiciomora',
'Master_Fvencimiento',
'Master_fultimo_cierre',
'Visa_Finiciomora',
'Visa_fultimo_cierre',
'Visa_mpagado',
'cpayroll_trx',
'mcaja_ahorro_adicional',
'mcomisiones',
'mcomisiones_mantenimiento',
'mcomisiones_otras',
'mpayroll',
'mrentabilidad',
'mtransferencias_recibidas']
